In [ ]:
:local-repo /opt/spark/repo

res2: String = Repo changed to /opt/spark/repo!


Repo changed to /opt/spark/repo!

In [ ]:
:dp org.bdgenomics.adam % adam-core % 0.16.0
- org.apache.hadoop % hadoop-client %   _
- org.apache.spark  %     _         %   _
- org.scala-lang    %     _         %   _
- org.scoverage     %     _         %   _

globalScope.jars: Array[String] = [Ljava.lang.String;@9965a0d
res5: List[String] = List(/home/ddecap/.m2/repository/org/apache/bcel/bcel/5.2/bcel-5.2.jar, /home/ddecap/.m2/repository/com/beust/jcommander/1.27/jcommander-1.27.jar, /home/ddecap/.m2/repository/com/twitter/parquet-generator/1.6.0rc4/parquet-generator-1.6.0rc4.jar, /home/ddecap/.m2/repository/org/apache/commons/commons-jexl/2.1.1/commons-jexl-2.1.1.jar, /home/ddecap/.m2/repository/org/codehaus/jackson/jackson-mapper-asl/1.9.13/jackson-mapper-asl-1.9.13.jar, /home/ddecap/.m2/repository/org/apache/commons/commons-compress/1.4.1/commons-compress-1.4.1.jar, /home/ddecap/.m2/repository/org/slf4j/slf4j-log4j12/1.7.5/slf4j-log4j12-1.7.5.jar, /home/ddecap/.m2/repository/commons-cli/commons-cli/1.2/commons-cli-1.2.jar, /home/ddecap/....

In [ ]:
reset(lastChanges = _.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
                   .set("spark.kryo.registrator", "org.bdgenomics.adam.serialization.ADAMKryoRegistrator")
                   .set("spark.kryoserializer.buffer.mb", "16")
                   .set("spark.kryo.referenceTracking", "true")
                   .setAppName("ADAM 1000genomes")
                   .set("spark.executor.memory", "8g")
      )

In [ ]:
import org.bdgenomics.adam.rdd.ADAMContext
import org.bdgenomics.adam.rdd.ADAMContext._
import org.apache.spark.rdd.RDD
import org.bdgenomics.adam.models.VariantContext
import org.bdgenomics.adam.rdd.ADAMContext
import org.bdgenomics.adam.rich.RichVariant
import org.bdgenomics.formats.avro._
import scala.annotation.tailrec
import org.apache.hadoop.fs.{FileSystem, Path}
import org.apache.hadoop.conf.Configuration 

@transient val ac = new ADAMContext(sc)

import org.bdgenomics.adam.rdd.ADAMContext
import org.bdgenomics.adam.rdd.ADAMContext._
import org.apache.spark.rdd.RDD
import org.bdgenomics.adam.models.VariantContext
import org.bdgenomics.adam.rdd.ADAMContext
import org.bdgenomics.adam.rich.RichVariant
import org.bdgenomics.formats.avro._
import scala.annotation.tailrec
import org.apache.hadoop.fs.{FileSystem, Path}
import org.apache.hadoop.conf.Configuration
ac: org.bdgenomics.adam.rdd.ADAMContext = org.bdgenomics.adam.rdd.ADAMContext@979171


org.bdgenomics.adam.rdd.ADAMContext@979171

In [ ]:
val wd = "/seq/cellLines/CCLE/50samples"
val rnaFile = wd + "/ALL-SIL-RNA-08.exomefiltered.vcf.annotated.vcf"
val wxsFile = wd + "/ALL-SIL-DNA-08.exomefiltered.vcf.annotated.vcf"
val rna = ac.loadGenotypes(rnaFile)
val rnaMap = rna.map(v => (v.getSampleId()+"@"+v.getVariant().getContig().getContigName()+"+"+v.getVariant().getStart().toString
                         +":"+v.getVariant().getReferenceAllele()+">"+v.getVariant().getAlternateAllele(), v))
val wxs = ac.loadGenotypes(wxsFile) 
val wxsMap = wxs.map(v => (v.getSampleId()+"@"+v.getVariant().getContig().getContigName()+"+"+v.getVariant().getStart().toString
                         +":"+v.getVariant().getReferenceAllele()+">"+v.getVariant().getAlternateAllele(), v))

wd: String = /seq/cellLines/CCLE/50samples
rnaFile: String = /seq/cellLines/CCLE/50samples/ALL-SIL-RNA-08.exomefiltered.vcf.annotated.vcf
wxsFile: String = /seq/cellLines/CCLE/50samples/ALL-SIL-DNA-08.exomefiltered.vcf.annotated.vcf
rna: org.apache.spark.rdd.RDD[org.bdgenomics.formats.avro.Genotype] = MapPartitionsRDD[2] at flatMap at ADAMContext.scala:464
rnaMap: org.apache.spark.rdd.RDD[(String, org.bdgenomics.formats.avro.Genotype)] = MapPartitionsRDD[3] at map at <console>:67
wxs: org.apache.spark.rdd.RDD[org.bdgenomics.formats.avro.Genotype] = MapPartitionsRDD[6] at flatMap at ADAMContext.scala:464
wxsMap: org.apache.spark.rdd.RDD[(String, org.bdgenomics.formats.avro.Genotype)] = MapPartitionsRDD[7] at map at <console>:70


MapPartitionsRDD[7] at map at <console>:70

In [ ]:
val coGroup = wxsMap.cogroup(rnaMap)
val both = coGroup.filter(v => (v._2._1.size >= 1 && v._2._2.size >= 1)).map(v => (v._2._1 ++ v._2._2).
              maxBy(_.getVariantCallingAnnotations().getVariantCallErrorProbability())).cache()
val wxs = coGroup.filter(v => (v._2._1.size >= 1 && v._2._2.size == 0)).map(v => v._2._1.
              maxBy(_.getVariantCallingAnnotations().getVariantCallErrorProbability())).cache()
val rna = coGroup.filter(v => (v._2._2.size >= 1 && v._2._1.size == 0)).map(v => v._2._2.
              maxBy(_.getVariantCallingAnnotations().getVariantCallErrorProbability())).cache()

coGroup: org.apache.spark.rdd.RDD[(String, (Iterable[org.bdgenomics.formats.avro.Genotype], Iterable[org.bdgenomics.formats.avro.Genotype]))] = MapPartitionsRDD[9] at cogroup at <console>:77
both: org.apache.spark.rdd.RDD[org.bdgenomics.formats.avro.Genotype] = MapPartitionsRDD[11] at map at <console>:78
wxs: org.apache.spark.rdd.RDD[org.bdgenomics.formats.avro.Genotype] = MapPartitionsRDD[13] at map at <console>:80
rna: org.apache.spark.rdd.RDD[org.bdgenomics.formats.avro.Genotype] = MapPartitionsRDD[15] at map at <console>:82


MapPartitionsRDD[15] at map at <console>:82

In [ ]:
val lnOf2 = math.log(2)
val DPstep = math.floor(math.log(math.max(rna.map(g => g.getReadDepth()).max(), wxs.map(g => g.getReadDepth()).max())) / lnOf2)
val Qstep = math.floor(math.log(math.max(rna.map(g => g.getVariantCallingAnnotations().getVariantCallErrorProbability()).max(), 
                                       wxs.map(g => g.getVariantCallingAnnotations().getVariantCallErrorProbability()).max())) / lnOf2)

lnOf2: Double = 0.6931471805599453
DPstep: Double = 12.0
Qstep: Double = 16.0


16.0

In [ ]:
val dp = 10
val gq = 30
val qual = 30

dp: Int = 10
gq: Int = 30
qual: Int = 30


30

In [ ]:
case class TabularRepr(threshold:Double, group: String, count:Long)
val qRange = List.tabulate(Qstep.toInt)(threshold => {
    val tmin = math.pow(2,threshold)
    val tmax = math.pow(2,threshold + 1)
    List(TabularRepr(tmin, "both",
    both.filter(x => (x.getReadDepth() > dp && x.getGenotypeQuality() > gq 
                      && x.getVariantCallingAnnotations().getVariantCallErrorProbability() > tmin && 
                      x.getVariantCallingAnnotations().getVariantCallErrorProbability() <= tmax)).count),
                             TabularRepr(tmin, "rna",
    rna.filter(x => (x.getReadDepth() > dp && x.getGenotypeQuality() > gq 
                     && x.getVariantCallingAnnotations().getVariantCallErrorProbability() > tmin && 
                      x.getVariantCallingAnnotations().getVariantCallErrorProbability() <= tmax)).count),
                             TabularRepr(tmin, "wxs",                                 
    wxs.filter(x => (x.getReadDepth() > dp && x.getGenotypeQuality() > gq 
                     && x.getVariantCallingAnnotations().getVariantCallErrorProbability() > tmin && 
                      x.getVariantCallingAnnotations().getVariantCallErrorProbability() <= tmax)).count))
    }).flatMap(x => x)
val dpRange = List.tabulate(DPstep.toInt)(threshold => {
    val tmin = math.pow(2,threshold)
    val tmax = math.pow(2,threshold + 1)
    List(TabularRepr(tmin, "both",
    both.filter(x => (x.getReadDepth() > tmin && x.getReadDepth() <= tmax && x.getGenotypeQuality() > gq 
                      && x.getVariantCallingAnnotations().getVariantCallErrorProbability() > qual)).count),
                             TabularRepr(tmin, "rna",
    rna.filter(x => (x.getReadDepth() > tmin && x.getReadDepth() <= tmax && x.getGenotypeQuality() > gq 
                     && x.getVariantCallingAnnotations().getVariantCallErrorProbability() > qual)).count),
                             TabularRepr(tmin, "wxs",                                 
    wxs.filter(x => (x.getReadDepth() > tmin && x.getReadDepth() <= tmax && x.getGenotypeQuality() > gq 
                     && x.getVariantCallingAnnotations().getVariantCallErrorProbability() > qual)).count))
    }).flatMap(x => x)
val gqRange = List.tabulate(100)(threshold => List(TabularRepr(threshold, "both",
    both.filter(x => (x.getReadDepth() > dp && x.getGenotypeQuality() == threshold 
                      && x.getVariantCallingAnnotations().getVariantCallErrorProbability() > qual)).count),
                             TabularRepr(threshold, "rna",
    rna.filter(x => (x.getReadDepth() > dp  && x.getGenotypeQuality() == threshold 
                     && x.getVariantCallingAnnotations().getVariantCallErrorProbability() > qual)).count),
                             TabularRepr(threshold, "wxs",                                 
    wxs.filter(x => (x.getReadDepth() > dp  && x.getGenotypeQuality() == threshold 
                     && x.getVariantCallingAnnotations().getVariantCallErrorProbability() > qual)).count))).flatMap(x => x)


defined class TabularRepr
qRange: List[TabularRepr] = List(TabularRepr(1.0,both,0), TabularRepr(1.0,rna,0), TabularRepr(1.0,wxs,0), TabularRepr(2.0,both,0), TabularRepr(2.0,rna,0), TabularRepr(2.0,wxs,0), TabularRepr(4.0,both,0), TabularRepr(4.0,rna,0), TabularRepr(4.0,wxs,0), TabularRepr(8.0,both,0), TabularRepr(8.0,rna,0), TabularRepr(8.0,wxs,0), TabularRepr(16.0,both,0), TabularRepr(16.0,rna,256), TabularRepr(16.0,wxs,14), TabularRepr(32.0,both,7), TabularRepr(32.0,rna,536), TabularRepr(32.0,wxs,321), TabularRepr(64.0,both,37), TabularRepr(64.0,rna,904), TabularRepr(64.0,wxs,638), TabularRepr(128.0,both,176), TabularRepr(128.0,rna,1413), TabularRepr(128.0,wxs,1197), TabularRepr(256.0,both,627), TabularRepr(256.0,rna,2104), TabularRepr(256.0,wxs,2345), TabularRepr(512.0,both,1607), Ta...

In [ ]:
val gqGraph = """
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(60, 30, 480, 330)
      var x = chart.addMeasureAxis("x", "threshold");
      x.ticks = 10
      chart.addLogAxis("y", "count");
      var s = chart.addSeries(["threshold", "group"], dimple.plot.line);
      s.lineWeight = 1;
      s.barGap = 0.05;
      chart.addLegend(550, 20, 100, 300, "left");
      chart.draw();
}
"""
val dpGraph = """
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(60, 30, 480, 330)
      var x = chart.addLogAxis("x", "threshold");
      x.ticks = 10
      x.overrideMin = 1
      chart.addMeasureAxis("y", "count");
      var s = chart.addSeries(["threshold", "group"], dimple.plot.line);
      s.lineWeight = 1;
      s.barGap = 0.05;
      chart.addLegend(550, 20, 100, 300, "left");
      chart.draw();
}
"""
val dpGraphPro = """
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(60, 30, 480, 330)
      var x = chart.addCategoryAxis("x", "threshold");
      x.ticks = 10
      x.overrideMin = 1
      chart.addPctAxis("y", "count");
      var s = chart.addSeries("group", dimple.plot.area);
      s.lineWeight = 1;
      s.barGap = 0.05;
      chart.addLegend(550, 20, 100, 300, "left");
      chart.draw();
}
"""
val splitBsGraph = """
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(80, 30, 480, 330)
      chart.addMeasureAxis("y", "count");
      chart.addCategoryAxis("x", "group");
      chart.addSeries(["reference", "alternate"], dimple.plot.bar);
      chart.addLegend(200, 10, 380, 20, "right");
      chart.draw();
}
"""
val mergedBsGraph = """
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(80, 30, 480, 330)
      chart.addMeasureAxis("y", "count");
      chart.addCategoryAxis("x", "group");
      chart.addSeries("baseChange", dimple.plot.bar);
      chart.addLegend(200, 10, 380, 20, "right");
      chart.draw();
}
"""

gqGraph: String = 
"
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(60, 30, 480, 330)
      var x = chart.addMeasureAxis("x", "threshold");
      x.ticks = 10
      chart.addLogAxis("y", "count");
      var s = chart.addSeries(["threshold", "group"], dimple.plot.line);
      s.lineWeight = 1;
      s.barGap = 0.05;
      chart.addLegend(550, 20, 100, 300, "left");
      chart.draw();
}
"
dpGraph: String = 
"
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(60, 30, 480, 330)
      var x = chart.addLogAxis("x", "threshold");
      x.ticks = 10
      x.overrideMin = 1
      chart.addMeasureAxis("y", "count");
      var s = chart.addSeries(["threshold", "group"], dimple.plot.line);
      s.lineWeight = 1;
      ...

function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
 chart.setBounds(80, 30, 480, 330)
 chart.addMeasureAxis("y", "count");
 chart.addCategoryAxis("x", "group");
 chart.addSeries("baseChange", dimple.plot.bar);
 chart.addLegend(200, 10, 380, 20, "right");
 chart.draw();
}

In [ ]:
widgets.DiyChart(gqRange, gqGraph, maxPoints=gqRange.length)

res7: notebook.front.widgets.DiyChart[List[TabularRepr]] = <DiyChart widget>


In [ ]:
widgets.DiyChart(dpRange, dpGraph, maxPoints=dpRange.length)

res8: notebook.front.widgets.DiyChart[List[TabularRepr]] = <DiyChart widget>


In [ ]:
widgets.DiyChart(qRange, dpGraph, maxPoints=qRange.length)

res9: notebook.front.widgets.DiyChart[List[TabularRepr]] = <DiyChart widget>


In [ ]:
case class BaseChangeRepr(group: String, reference:String, alternate:String, count:Long)
case class BaseChangeRepr2(group: String, baseChange:String, count:Long)
def isSingleNucleotideVariant(g: org.bdgenomics.formats.avro.Genotype): Boolean = {
    g.getVariant().getReferenceAllele().length == 1 && g.getVariant().getAlternateAllele().length == 1
  }
val baseChanges = List(
              both.filter(g => isSingleNucleotideVariant(g))
                .map(g => (g.getVariant().getReferenceAllele(), g.getVariant().getAlternateAllele()))
                .countByValue.map(x => BaseChangeRepr("both", x._1._1, x._1._2, x._2))
                .toList,
              rna.filter(g => isSingleNucleotideVariant(g))
                .map(g => (g.getVariant().getReferenceAllele(), g.getVariant().getAlternateAllele()))
                .countByValue.map(x => BaseChangeRepr("rna", x._1._1, x._1._2, x._2))
                .toList,
              wxs.filter(g => isSingleNucleotideVariant(g))
                .map(g => (g.getVariant().getReferenceAllele(), g.getVariant().getAlternateAllele()))
                .countByValue.map(x => BaseChangeRepr("wxs", x._1._1, x._1._2, x._2))
                .toList).flatMap(x => x)
val baseChanges2 = List(
              both.filter(g => isSingleNucleotideVariant(g))
                .map(g => (g.getVariant().getReferenceAllele(), g.getVariant().getAlternateAllele()))
                .countByValue.map(x => BaseChangeRepr2("both", x._1._1+">"+x._1._2, x._2))
                .toList,
              rna.filter(g => isSingleNucleotideVariant(g))
                .map(g => (g.getVariant().getReferenceAllele(), g.getVariant().getAlternateAllele()))
                .countByValue.map(x => BaseChangeRepr2("rna", x._1._1+">"+x._1._2, x._2))
                .toList,
              wxs.filter(g => isSingleNucleotideVariant(g))
                .map(g => (g.getVariant().getReferenceAllele(), g.getVariant().getAlternateAllele()))
                .countByValue.map(x => BaseChangeRepr2("wxs", x._1._1+">"+x._1._2, x._2))
                .toList).flatMap(x => x)
  

defined class BaseChangeRepr
defined class BaseChangeRepr2
isSingleNucleotideVariant: (g: org.bdgenomics.formats.avro.Genotype)Boolean
baseChanges: List[BaseChangeRepr] = List(BaseChangeRepr(both,C,G,523), BaseChangeRepr(both,T,G,307), BaseChangeRepr(both,T,C,2092), BaseChangeRepr(both,G,T,336), BaseChangeRepr(both,C,T,2109), BaseChangeRepr(both,G,C,542), BaseChangeRepr(both,C,A,330), BaseChangeRepr(both,T,A,197), BaseChangeRepr(both,A,C,348), BaseChangeRepr(both,A,G,2002), BaseChangeRepr(both,A,T,195), BaseChangeRepr(both,G,A,2142), BaseChangeRepr(rna,C,G,694), BaseChangeRepr(rna,T,G,532), BaseChangeRepr(rna,T,C,3061), BaseChangeRepr(rna,G,T,565), BaseChangeRepr(rna,C,T,2506), BaseChangeRepr(rna,G,C,714), BaseChangeRepr(rna,C,A,601), BaseChangeRepr(rna,T,A,383), BaseChangeRepr(rna,A,C,...

In [ ]:
widgets.DiyChart(baseChanges, splitBsGraph, maxPoints=baseChanges.length)

res21: notebook.front.widgets.DiyChart[List[BaseChangeRepr]] = <DiyChart widget>


In [ ]:
widgets.DiyChart(baseChanges2, mergedBsGraph, maxPoints=baseChanges2.length)

res22: notebook.front.widgets.DiyChart[List[BaseChangeRepr2]] = <DiyChart widget>
